RAG chatbot 
Version 1.0 

Import Libraries

In [2]:
# Imports
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain_classic.retrievers import MultiQueryRetriever
import chromadb

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Suppress ChromaDB telemetry warnings as they are useless as ChromaDB trying to send usage data but failing because it can't find its own version information
import logging
logging.getLogger("chromadb").setLevel(logging.ERROR)

# Jupyter-specific imports
from IPython.display import display, Markdown

# Set environment variable for protobuf
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

LOAD PDF

In [ ]:
# Load PDF
local_path = "TaskFlow.pdf"
if local_path:
    loader = UnstructuredPDFLoader(file_path=local_path, languages=["deu"])
    data = loader.load()
    print(f"PDF loaded successfully: {local_path}")
else:
    print("Upload documents")

#TODO: language detection!!!!

SPLIT TEXT INTO CHUNKS

In [ ]:
#Split text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(data)
print(f"Number of text chunks: {len(chunks)}")

CREATE VECTOR DATABASE

In [ ]:
# Create vector database
client = chromadb.PersistentClient(path="./chroma_data")
vector_db = Chroma.from_documents(
    documents=chunks,
    embedding=OllamaEmbeddings(model="nomic-embed-text"),
    collection_name="rag_chatbot_collection",
    client=client
)
print("Vector database created successfully.")
